# ESPN Fantasy Football Draft

### First use the code below to open ESPN on a FireFox Browser. Use it to log in and navigate to the draft

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

browser = webdriver.Firefox()

browser.get('https://www.espn.com')
# assert 'Yahoo' in browser.title

# elem = browser.find_element(By.NAME, 'p')  # Find the search box
# elem.send_keys('seleniumhq' + Keys.RETURN)

# browser.quit()

### Enter the Draft Lobby and go to 'Edit PreDraft Rankings' for the predraft rankings, which will be used for data

In [2]:
browser.current_url

'https://fantasy.espn.com/football/editdraftstrategy?leagueId=455161439'

Click "Show more" a few times until there are an adequate number of players for your draft... Let's say 300

In [3]:
from bs4 import BeautifulSoup
import copy
import pandas as pd

html=browser.page_source
soup=BeautifulSoup(html,'html.parser')
div = soup.find_all("div", "jsx-4130407500 table-stats-container")
cols = div[0].find_all("tr", {'class':"Table__sub-header Table__TR Table__even"})[0].find_all("th", {"class": "Table__TH"})
cols = [col.text for col in cols]
vals = div[0].find_all("tr", "Table__TR Table__TR--sm Table__odd")[0].find_all("td")
vals = [val.text for val in vals]
print("Number of Records:", len(div[0].find_all("tr", "Table__TR Table__TR--sm Table__odd")))
data = []
for i in range(0, len(div[0].find_all("tr", "Table__TR Table__TR--sm Table__odd"))):
    vals = div[0].find_all("tr", "Table__TR Table__TR--sm Table__odd")[i].find_all("td")
    vals = [val.text for val in vals]
    row = {}
    for j in range(0,len(vals)):
        row[cols[j]] = vals[j]
    data.append(copy.deepcopy(row))
projections = pd.DataFrame(data)

Number of Records: 300


In [4]:
projections

,,RANK,PLAYER,EXCLUDE,YDS,TD,INT,REC,FR,PA,YA,FPTS
0,,1,Justin JeffersonMinWR,,1498,9,0,116,0,0,0,322.3
1,,2,Ja'Marr ChaseCinWR,,1454,11,0,102,0,0,0,313.3
2,,3,Derrick HenryTenRB,,285,1,0,32,0,0,0,256.6
3,,4,Austin EkelerLACRB,,500,3,0,67,0,0,0,273.9
4,,5,Travis KelceKCTE,,1145,10,0,101,0,0,0,279
...,...,...,...,...,...,...,...,...,...,...,...,...
295,,296,Baker MayfieldTBQB,,0,0,0,0,0,0,0,138.1
296,,297,Falcons D/STAtlD/ST,,0,0,13,0,9,375,5921,98.9
297,,298,Chiefs D/STKCD/ST,,0,0,14,0,9,395,6021,84.6
298,,299,Broncos D/STDenD/ST,,0,0,13,0,9,382,5948,86.4


In [5]:
projections.loc[projections['PLAYER'].str.endswith("WR"), 'position'] = "WR"
projections.loc[projections['PLAYER'].str.endswith("RB"), 'position'] = "RB"
projections.loc[projections['PLAYER'].str.endswith("QB"), 'position'] = "QB"
projections.loc[projections['PLAYER'].str.endswith("TE"), 'position'] = "TE"
projections.loc[projections['PLAYER'].str.endswith("D/ST"), 'position'] = "D"
projections.loc[projections['PLAYER'].str.endswith("K"), 'position'] = "K"
projections.loc[projections['position'].isin(["WR", "RB", "QB", "TE"]), 'PLAYER'] = projections.loc[projections['position'].isin(["WR", "RB", "QB", "TE"]), 'PLAYER'].str[:-2]
projections.loc[projections['position'].isin(["D"]), 'PLAYER'] = projections.loc[projections['position'].isin(["D"]), 'PLAYER'].str[:-4]
projections.loc[projections['position'].isin(["K"]), 'PLAYER'] = projections.loc[projections['position'].isin(["K"]), 'PLAYER'].str[:-1]

In [6]:
projections

,,RANK,PLAYER,EXCLUDE,YDS,TD,INT,REC,FR,PA,YA,FPTS,position
0,,1,Justin JeffersonMin,,1498,9,0,116,0,0,0,322.3,WR
1,,2,Ja'Marr ChaseCin,,1454,11,0,102,0,0,0,313.3,WR
2,,3,Derrick HenryTen,,285,1,0,32,0,0,0,256.6,RB
3,,4,Austin EkelerLAC,,500,3,0,67,0,0,0,273.9,RB
4,,5,Travis KelceKC,,1145,10,0,101,0,0,0,279,TE
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,,296,Baker MayfieldTB,,0,0,0,0,0,0,0,138.1,QB
296,,297,Falcons D/STAtl,,0,0,13,0,9,375,5921,98.9,D
297,,298,Chiefs D/STKC,,0,0,14,0,9,395,6021,84.6,D
298,,299,Broncos D/STDen,,0,0,13,0,9,382,5948,86.4,D


In [7]:
projections.loc[projections['PLAYER'].str.endswith("Min"), 'team'] = "Min"
projections.loc[projections['PLAYER'].str.endswith("Cin"), 'team'] = "Cin"
projections.loc[projections['PLAYER'].str.endswith("Ten"), 'team'] = "Ten"
projections.loc[projections['PLAYER'].str.endswith("LAC"), 'team'] = "LAC"
projections.loc[projections['PLAYER'].str.endswith("Atl"), 'team'] = "Atl"
projections.loc[projections['PLAYER'].str.endswith("Den"), 'team'] = "Den"
projections.loc[projections['PLAYER'].str.endswith("Hou"), 'team'] = "Hou"
projections.loc[projections['PLAYER'].str.endswith("LAR"), 'team'] = "LAR"
projections.loc[projections['PLAYER'].str.endswith("Mia"), 'team'] = "Mia"
projections.loc[projections['PLAYER'].str.endswith("Cle"), 'team'] = "Cle"
projections.loc[projections['PLAYER'].str.endswith("NYG"), 'team'] = "NYG"
projections.loc[projections['PLAYER'].str.endswith("Buf"), 'team'] = "Buf"
projections.loc[projections['PLAYER'].str.endswith("Dal"), 'team'] = "Dal"
projections.loc[projections['PLAYER'].str.endswith("NYJ"), 'team'] = "NYJ"
projections.loc[projections['PLAYER'].str.endswith("Phi"), 'team'] = "Phi"
projections.loc[projections['PLAYER'].str.endswith("Det"), 'team'] = "Det"
projections.loc[projections['PLAYER'].str.endswith("Ind"), 'team'] = "Ind"
projections.loc[projections['PLAYER'].str.endswith("Sea"), 'team'] = "Sea"
projections.loc[projections['PLAYER'].str.endswith("Jax"), 'team'] = "Jax"
projections.loc[projections['PLAYER'].str.endswith("Pit"), 'team'] = "Pit"
projections.loc[projections['PLAYER'].str.endswith("Bal"), 'team'] = "Bal"
projections.loc[projections['PLAYER'].str.endswith("Car"), 'team'] = "Car"
projections.loc[projections['PLAYER'].str.endswith("Wsh"), 'team'] = "Wsh"
projections.loc[projections['PLAYER'].str.endswith("Ari"), 'team'] = "Ari"
projections.loc[projections['PLAYER'].str.endswith("Chi"), 'team'] = "Chi"

projections.loc[projections['PLAYER'].str.endswith("KC"), 'team'] = "KC"
projections.loc[projections['PLAYER'].str.endswith("TB"), 'team'] = "TB"
projections.loc[projections['PLAYER'].str.endswith("SF"), 'team'] = "SF"
projections.loc[projections['PLAYER'].str.endswith("LV"), 'team'] = "LV"
projections.loc[projections['PLAYER'].str.endswith("NO"), 'team'] = "NO"
projections.loc[projections['PLAYER'].str.endswith("GB"), 'team'] = "GB"
projections.loc[projections['PLAYER'].str.endswith("NE"), 'team'] = "NE"

projections.loc[projections['team'].isin(["Min", "Cin", "Ten", "LAC", "Atl", "Den", "Hou", "LAR", "Mia", "Cle", "NYG", "Buf", "Dal", "NYJ", "Phi", "Det", "Ind", "Sea", "Jax", "Pit", "Bal", "Car", "Wsh", "Ari", "Chi"]), 'PLAYER'] = projections.loc[projections['team'].isin(["Min", "Cin", "Ten", "LAC", "Atl", "Den", "Hou", "LAR", "Mia", "Cle", "NYG", "Buf", "Dal", "NYJ", "Phi", "Det", "Ind", "Sea", "Jax", "Pit", "Bal", "Car", "Wsh", "Ari", "Chi"]), 'PLAYER'].str[:-3]
projections.loc[projections['team'].isin(["KC", "TB", "SF", "LV", "NO", "GB", "NE"]), 'PLAYER'] = projections.loc[projections['team'].isin(["KC", "TB", "SF", "LV", "NO", "GB", "NE"]), 'PLAYER'].str[:-2]

In [8]:
projections

,,RANK,PLAYER,EXCLUDE,YDS,TD,INT,REC,FR,PA,YA,FPTS,position,team
0,,1,Justin Jefferson,,1498,9,0,116,0,0,0,322.3,WR,Min
1,,2,Ja'Marr Chase,,1454,11,0,102,0,0,0,313.3,WR,Cin
2,,3,Derrick Henry,,285,1,0,32,0,0,0,256.6,RB,Ten
3,,4,Austin Ekeler,,500,3,0,67,0,0,0,273.9,RB,LAC
4,,5,Travis Kelce,,1145,10,0,101,0,0,0,279,TE,KC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,,296,Baker Mayfield,,0,0,0,0,0,0,0,138.1,QB,TB
296,,297,Falcons D/ST,,0,0,13,0,9,375,5921,98.9,D,Atl
297,,298,Chiefs D/ST,,0,0,14,0,9,395,6021,84.6,D,KC
298,,299,Broncos D/ST,,0,0,13,0,9,382,5948,86.4,D,Den


In [9]:
projections.loc[projections['PLAYER'].str.endswith("Q"), 'PLAYER'] = projections.loc[projections['PLAYER'].str.endswith("Q"), 'PLAYER'].str[:-1]
projections.loc[projections['PLAYER'].str.endswith("O"), 'PLAYER'] = projections.loc[projections['PLAYER'].str.endswith("O"), 'PLAYER'].str[:-1]
projections.loc[projections['PLAYER'].str.endswith("SSPD"), 'PLAYER'] = projections.loc[projections['PLAYER'].str.endswith("SSPD"), 'PLAYER'].str[:-4]

In [10]:
projections

,,RANK,PLAYER,EXCLUDE,YDS,TD,INT,REC,FR,PA,YA,FPTS,position,team
0,,1,Justin Jefferson,,1498,9,0,116,0,0,0,322.3,WR,Min
1,,2,Ja'Marr Chase,,1454,11,0,102,0,0,0,313.3,WR,Cin
2,,3,Derrick Henry,,285,1,0,32,0,0,0,256.6,RB,Ten
3,,4,Austin Ekeler,,500,3,0,67,0,0,0,273.9,RB,LAC
4,,5,Travis Kelce,,1145,10,0,101,0,0,0,279,TE,KC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,,296,Baker Mayfield,,0,0,0,0,0,0,0,138.1,QB,TB
296,,297,Falcons D/ST,,0,0,13,0,9,375,5921,98.9,D,Atl
297,,298,Chiefs D/ST,,0,0,14,0,9,395,6021,84.6,D,KC
298,,299,Broncos D/ST,,0,0,13,0,9,382,5948,86.4,D,Den


In [11]:
projections['FPTS'] = projections['FPTS'].astype(float)
projections = projections.sort_values(by="FPTS", ascending=False)

### Prepare the draft State

In [12]:
class DraftState:
    def __init__(self, rosters, turns, freeagents, playerjm=None):
        self.rosters = rosters
        self.turns = turns
        self.freeagents = freeagents
        self.playerJustMoved = playerjm

class NflPlayer:
    def __init__(self, name, team, position, points):
        self.name = name
        self.team = team
        self.position = position
        self.points = points
        
    def __repr__(self):
        return "|".join([self.name, self.team, self.position, str(self.points)])

In [13]:
import numpy as np

def GetResult(self, playerjm):
    """ Get the game result from the viewpoint of playerjm.
    """
    if playerjm is None: return 0
    
    pos_wgts = {
        ("QB"): [.7, .3],
        ("WR"): [.7, .7, .5, .2],
        ("RB"): [.7, .7, .5, .2],
        ("TE"): [.6, .4],
        ("RB", "WR", "TE"): [.6, .4],
        ("D"): [.6, .1],
        ("K"): [.5, .1]
    }

    result = 0
    # map the drafted players to the weights
    for p in self.rosters[playerjm]:
        max_wgt, _, max_pos, old_wgts = max(
            ((wgts[0], -len(lineup_pos), lineup_pos, wgts) for lineup_pos, wgts in pos_wgts.items()
                if p.position in lineup_pos),
            default=(0, 0, (), []))
        if max_wgt > 0:
            result += max_wgt * p.points
            old_wgts.pop(0)
            if not old_wgts:
                pos_wgts.pop(max_pos)
                
    # map the remaining weights to the top three free agents
    for pos, wgts in pos_wgts.items():
        result += np.mean([p.points for p in self.freeagents if p.position in pos][:3]) * sum(wgts)
        
    return result
        
DraftState.GetResult = GetResult

In [14]:
def GetMoves(self):
    """ Get all possible moves from this state.
    """
    pos_max = {"QB": 2, "WR": 6, "RB": 6, "TE": 2, "D": 1, "K": 1}

    if len(self.turns) == 0: return []

    roster_positions = np.array([p.position for p in self.rosters[self.turns[0]]], dtype=str)
    moves = [pos for pos, max_ in pos_max.items() if np.sum(roster_positions == pos) < max_]
    return moves

DraftState.GetMoves = GetMoves

def DoMove(self, move, player_selected = "", available = "Y"):
    """ Update a state by carrying out the given move.
        Must update playerJustMoved.
    """
    # print(move)
    if move != "manual":
        player = next(p for p in self.freeagents if p.position == move)
    elif available == "Y":
        player = next(x for x in state.freeagents if x.name == player_selected )
    # print(player)
    rosterId = self.turns.pop(0)
    if available == "Y":
        self.freeagents.remove(player)
        self.rosters[rosterId].append(player)
    else:
        team = input(f"{player_selected}'s team")
        position = input(f"{player_selected}'s postion")
        player_selected = NflPlayer(player_selected, team, position, 0)

        self.rosters[rosterId].append(player_selected)
    
    self.playerJustMoved = rosterId
    
DraftState.DoMove = DoMove

def Clone(self):
    """ Create a deep clone of this game state.
    """
    rosters = list(map(lambda r: r[:], self.rosters))
    st = DraftState(rosters, self.turns[:], self.freeagents[:],
            self.playerJustMoved)
    return st

DraftState.Clone = Clone

In [15]:
# This is a very simple implementation of the UCT Monte Carlo Tree Search algorithm in Python 2.7.
# The function UCT(rootstate, itermax, verbose = False) is towards the bottom of the code.
# It aims to have the clearest and simplest possible code, and for the sake of clarity, the code
# is orders of magnitude less efficient than it could be made, particularly by using a 
# state.GetRandomMove() or state.DoRandomRollout() function.
# 
# Written by Peter Cowling, Ed Powley, Daniel Whitehouse (University of York, UK) September 2012.
# 
# Licence is granted to freely use and distribute for any sensible/legal purpose so long as this comment
# remains in any distributed code.
# 
# For more information about Monte Carlo Tree Search check out our web site at www.mcts.ai

from math import *
import random
import time

class Node:
    """ A node in the game tree. Note wins is always from the viewpoint of playerJustMoved.
        Crashes if state not specified.
    """
    def __init__(self, move = None, parent = None, state = None):
        self.move = move # the move that got us to this node - "None" for the root node
        self.parentNode = parent # "None" for the root node
        self.childNodes = []
        self.wins = 0
        self.visits = 0
        self.untriedMoves = state.GetMoves() # future child nodes
        self.playerJustMoved = state.playerJustMoved # the only part of the state that the Node needs later
        
    def UCTSelectChild(self):
        """ Use the UCB1 formula to select a child node. Often a constant UCTK is applied so we have
            lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits to vary the amount of
            exploration versus exploitation.
        """
        UCTK = 200
        s = sorted(self.childNodes, key = lambda c: c.wins/c.visits + UCTK * sqrt(2*log(self.visits)/c.visits))[-1]
        return s
    
    def AddChild(self, m, s):
        """ Remove m from untriedMoves and add a new child node for this move.
            Return the added child node
        """
        n = Node(move = m, parent = self, state = s)
        self.untriedMoves.remove(m)
        self.childNodes.append(n)
        return n
    
    def Update(self, result):
        """ Update this node - one additional visit and result additional wins. result must be from the viewpoint of playerJustmoved.
        """
        self.visits += 1
        self.wins += result


def UCT(rootstate, itermax, verbose = False):
    """ Conduct a UCT search for itermax iterations starting from rootstate.
        Return the best move from the rootstate.
    """

    rootnode = Node(state = rootstate)

    for i in range(itermax):
        node = rootnode
        state = rootstate.Clone()

        # Select
        while node.untriedMoves == [] and node.childNodes != []: # node is fully expanded and non-terminal
            node = node.UCTSelectChild()
            state.DoMove(node.move)

        # Expand
        if node.untriedMoves != []: # if we can expand (i.e. state/node is non-terminal)
            m = random.choice(node.untriedMoves) 
            state.DoMove(m)
            node = node.AddChild(m,state) # add child and descend tree

        # Rollout - this can often be made orders of magnitude quicker using a state.GetRandomMove() function
        while state.GetMoves() != []: # while state is non-terminal
            state.DoMove(random.choice(state.GetMoves()))

        # Backpropagate
        while node != None: # backpropagate from the expanded node and work back to the root node
            node.Update(state.GetResult(node.playerJustMoved)) # state is terminal. Update node with result from POV of node.playerJustMoved
            node = node.parentNode

    return sorted(rootnode.childNodes, key = lambda c: c.visits)[-1].move # return the move that was most visited

### Now Enter the Draft

In [16]:
import os

nfl_players = projections[['PLAYER', 'team', 'position', "FPTS"]].copy()
nfl_players['FPTS'] = nfl_players['FPTS'].astype(float)
freeagents = [NflPlayer(*p) for p in nfl_players.itertuples(index=False, name=None)]

num_competitors = 12
rosters = [[] for _ in range(num_competitors)] # empty rosters to start with

num_rounds = 16
turns = []
# generate turns by snake order
for i in range(num_rounds):
    turns += reversed(range(num_competitors)) if i % 2 else range(num_competitors)
state = DraftState(rosters, turns, freeagents)
iterations = 1000

my_draft_spot = 12

pick_nbr = 0
while state.GetMoves() != []:
    pick_nbr += 1
    # print(pick_nbr)
    # move = UCT(state, iterations)
    # print(move)
    # print("Team", state.turns[0], ":", move, next(p for p in state.freeagents if p.position == move))
    # state.DoMove(move)
    if my_draft_spot == state.turns[0] + 1:
        start_time = time.time()
        move = UCT(state, iterations)
        print("Timing:", round(time.time() - start_time), "seconds")
        print("This is your best pick!", move, next(p for p in state.freeagents if p.position == move))
        os.system('say "Its your pick!"')
        os.system(f'say "{move}, {next(p for p in state.freeagents if p.position == move)}"')
    
    waiting_for_pick = True

    while waiting_for_pick:
        html=browser.page_source
        soup=BeautifulSoup(html,'html.parser')
        div = soup.find_all("div", "jsx-553213854 overflow-hidden w-100")
        data = []
        for i in range(0, len(div[0].find_all("li", {'class':"jsx-2093861861 b--solid bw1 bg-clr-white pick-message__container flex items-end truncate message"}))):
            row = {}
            t = div[0].find_all("li", {'class':"jsx-2093861861 b--solid bw1 bg-clr-white pick-message__container flex items-end truncate message"})[i].text
            row['name'] = t.split(" /")[0]
            row['pick'] = i + 1
            data.append(copy.deepcopy(row))
        picks = pd.DataFrame(data)
        if data:
            if pick_nbr in picks['pick'].to_list():
                temp = picks[picks['pick'] == pick_nbr]
                # print(temp['name'].values[0])
                player_selected = temp['name'].values[0]
                turn = state.turns[0]
                try:
                    state.DoMove("manual", player_selected = player_selected)
                    print(f"Team {turn + 1 if my_draft_spot != turn + 1 else 'Stassen'}: ", player_selected)
                except:
                    try:
                        print(f"{player_selected} Not found, try again")
                        player_manually_selected = input(f"Team {state.turns[0]}")
                        state.DoMove("manual", player_selected = player_manually_selected)
                        print(f"Team {turn + 1 if my_draft_spot != turn + 1 else 'Stassen'}: ", player_manually_selected)
                    except:
                        print(f"{player_selected} Not found, adding manually")
                        print(f"Team {turn + 1 if my_draft_spot != turn + 1 else 'Stassen'}: ", player_selected)
                        state.DoMove("manual", player_selected = player_selected, available="N")
                waiting_for_pick = False
            else:
                time.sleep(1)
        else:
            time.sleep(1)


    
    # else:
    #     try:
    #         player_selected = input(f"Team {state.turns[0] + 1}")
    #         print(f"Team {state.turns[0] + 1}: ", player_selected)
    #         state.DoMove("manual", player_selected = player_selected)
    #     except:
    #         try:
    #             print("Player Not found, try again")
    #             player_selected = input(f"Team {state.turns[0]}")
    #             print(f"Team {state.turns[0] + 1}: ", player_selected)
    #             state.DoMove("manual", player_selected = player_selected)
    #         except:
    #             print("Player Not found, adding manually")
    #             print(f"Team {state.turns[0] + 1}: ", player_selected)
    #             state.DoMove("manual", player_selected = player_selected, available="N")

Team 1:  Austin Ekeler
Team 2:  Justin Jefferson
Team 3:  Travis Kelce
Team 4:  Christian McCaffrey
Team 5:  Ja'Marr Chase
Team 6:  Saquon Barkley
Team 7:  Cooper Kupp
Team 8:  Tyreek Hill
Team 9:  Bijan Robinson
Team 10:  Stefon Diggs
Team 11:  Davante Adams
Timing: 11 seconds
This is your best pick! QB Patrick Mahomes|KC|QB|370.3
Team Stassen:  Patrick Mahomes
Timing: 12 seconds
This is your best pick! RB Derrick Henry|Ten|RB|256.6
Team Stassen:  Derrick Henry
Team 11:  CeeDee Lamb
Team 10:  Tony Pollard
Team 9:  Nick Chubb
Team 8:  Garrett Wilson
Team 7:  Josh Jacobs
Team 6:  Amon-Ra St. Brown
Team 5:  A.J. Brown
Team 4:  Chris Olave
Team 3:  Jaylen Waddle
Team 2:  Josh Allen
Team 1:  Tee Higgins
Team 1:  DeVonta Smith
Team 2:  DK Metcalf
Team 3:  Rhamondre Stevenson
Team 4:  Lamar Jackson
Team 5:  Jonathan Taylor
Team 6:  Joe Mixon
Team 7:  Travis Etienne Jr.
Team 8:  Jalen Hurts
Team 9:  Calvin Ridley
Team 10:  Jahmyr Gibbs
Team 11:  Najee Harris
Timing: 10 seconds
This is your be

### Draft is now complete, close the browser

In [17]:
browser.quit()

### Evaluate Performance

In [18]:
final_teams = pd.DataFrame({"Team " + str(i + 1 if my_draft_spot != i + 1 else 'Stassen'): r for i, r in enumerate(state.rosters)})
final_teams

,Team 1,Team 2,Team 3,Team 4,Team 5,Team 6,Team 7,Team 8,Team 9,Team 10,Team 11,Team Stassen
0,Austin Ekeler|LAC|RB|273.9,Justin Jefferson|Min|WR|322.3,Travis Kelce|KC|TE|279.0,Christian McCaffrey|SF|RB|272.0,Ja'Marr Chase|Cin|WR|313.3,Saquon Barkley|NYG|RB|253.9,Cooper Kupp|LAR|WR|310.6,Tyreek Hill|Mia|WR|293.8,Bijan Robinson|Atl|RB|256.0,Stefon Diggs|Buf|WR|264.3,Davante Adams|LV|WR|287.7,Patrick Mahomes|KC|QB|370.3
1,Tee Higgins|Cin|WR|228.3,Josh Allen|Buf|QB|357.5,Jaylen Waddle|Mia|WR|230.9,Chris Olave|NO|WR|221.4,A.J. Brown|Phi|WR|227.5,Amon-Ra St. Brown|Det|WR|243.3,Josh Jacobs|LV|RB|249.5,Garrett Wilson|NYJ|WR|250.3,Nick Chubb|Cle|RB|238.4,Tony Pollard|Dal|RB|244.7,CeeDee Lamb|Dal|WR|264.0,Derrick Henry|Ten|RB|256.6
2,DeVonta Smith|Phi|WR|218.9,DK Metcalf|Sea|WR|228.7,Rhamondre Stevenson|NE|RB|206.5,Lamar Jackson|Bal|QB|327.7,Jonathan Taylor|Ind|RB|229.7,Joe Mixon|Cin|RB|226.5,Travis Etienne Jr.|Jax|RB|215.0,Jalen Hurts|Phi|QB|337.5,Calvin Ridley|Jax|WR|202.5,Jahmyr Gibbs|Det|RB|214.9,Najee Harris|Pit|RB|214.9,Mark Andrews|Bal|TE|205.8
3,Alexander Mattison|Min|RB|199.4,Rachaad White|TB|RB|201.9,James Conner|Ari|RB|203.3,Dameon Pierce|Hou|RB|203.1,Christian Watson|GB|WR|208.5,Justin Herbert|LAC|QB|296.0,Joe Burrow|Cin|QB|336.0,Breece Hall|NYJ|RB|201.9,Amari Cooper|Cle|WR|209.0,Keenan Allen|LAC|WR|215.8,Deebo Samuel|SF|WR|210.7,Aaron Jones|GB|RB|220.2
4,Justin Fields|Chi|QB|308.9,Darren Waller|NYG|TE|186.1,Chris Godwin|TB|WR|202.7,DJ Moore|Chi|WR|203.1,T.J. Hockenson|Min|TE|192.5,Mike Williams|LAC|WR|200.9,DeAndre Hopkins|Ten|WR|204.6,Kenneth Walker III|Sea|RB|198.4,Miles Sanders|Car|RB|203.1,Mike Evans|TB|WR|205.1,Alvin Kamara|NO|RB|196.5,David Montgomery|Det|RB|196.4
5,Brandon Aiyuk|SF|WR|186.7,Isiah Pacheco|KC|RB|192.6,Christian Kirk|Jax|WR|189.3,Kyle Pitts|Atl|TE|160.5,James Cook|Buf|RB|195.8,George Kittle|SF|TE|165.8,Terry McLaurin|Wsh|WR|198.8,Tyler Lockett|Sea|WR|197.8,Drake London|Atl|WR|193.4,Trevor Lawrence|Jax|QB|282.3,Diontae Johnson|Pit|WR|204.7,Cam Akers|LAR|RB|196.2
6,Dallas Goedert|Phi|TE|147.7,Marquise Brown|Ari|WR|195.8,Jerry Jeudy|Den|WR|194.4,George Pickens|Pit|WR|180.6,Deshaun Watson|Cle|QB|297.3,Javonte Williams|Den|RB|189.6,Pat Freiermuth|Pit|TE|148.7,Brandin Cooks|Dal|WR|189.5,Jahan Dotson|Wsh|WR|173.0,Khalil Herbert|Chi|RB|167.4,Evan Engram|Jax|TE|149.3,Courtland Sutton|Den|WR|186.0
7,Zay Flowers|Bal|WR|160.1,Jamaal Williams|NO|RB|137.9,Samaje Perine|Den|RB|156.4,Jordan Addison|Min|WR|167.2,Jaxon Smith-Njigba|Sea|WR|161.3,AJ Dillon|GB|RB|171.8,Michael Pittman Jr.|Ind|WR|180.8,Antonio Gibson|Wsh|RB|148.6,David Njoku|Cle|TE|139.4,D'Andre Swift|Phi|RB|162.9,Dalvin Cook|NYJ|RB|155.4,J.K. Dobbins|Bal|RB|180.9
8,Quentin Johnston|LAC|WR|151.7,Gabe Davis|Buf|WR|162.2,Michael Thomas|NO|WR|163.9,Skyy Moore|KC|WR|167.8,Brian Robinson Jr.|Wsh|RB|150.0,Treylon Burks|Ten|WR|167.8,Cole Kmet|Chi|TE|135.8,Greg Dulcich|Den|TE|110.2,Dak Prescott|Dal|QB|281.8,Elijah Moore|Cle|WR|141.9,Kirk Cousins|Min|QB|285.0,Jakobi Meyers|LV|WR|173.7
9,Rashaad Penny|Phi|RB|127.4,Jeff Wilson Jr.|Mia|RB|151.8,Anthony Richardson|Ind|QB|243.9,Zach Charbonnet|Sea|RB|128.7,Rondale Moore|Ari|WR|167.0,Tua Tagovailoa|Mia|QB|257.6,Daniel Jones|NYG|QB|269.0,Chigoziem Okonkwo|Ten|TE|118.6,Odell Beckham Jr.|Bal|WR|151.3,Dalton Kincaid|Buf|TE|120.7,JuJu Smith-Schuster|NE|WR|167.1,Tyler Higbee|LAR|TE|143.1


In [19]:
# Which teams have highest projected points
for col in final_teams.columns:
    print(col, round(final_teams[col].apply(lambda x: x.points).sum()))

Team 1 2671
Team 2 2837
Team 3 2764
Team 4 2866
Team 5 3038
Team 6 3100
Team 7 3043
Team 8 2740
Team 9 2989
Team 10 2734
Team 11 3086
Team Stassen 3070


In [20]:
final_teams['Team Stassen']

0       Patrick Mahomes|KC|QB|370.3
1        Derrick Henry|Ten|RB|256.6
2         Mark Andrews|Bal|TE|205.8
3           Aaron Jones|GB|RB|220.2
4     David Montgomery|Det|RB|196.4
5            Cam Akers|LAR|RB|196.2
6     Courtland Sutton|Den|WR|186.0
7         J.K. Dobbins|Bal|RB|180.9
8         Jakobi Meyers|LV|WR|173.7
9         Tyler Higbee|LAR|TE|143.1
10          Geno Smith|Sea|QB|271.6
11        Steelers D/ST|Pit|D|133.8
12        Allen Lazard|NYJ|WR|150.1
13       Joshua Kelley|LAC|RB|107.7
14       Cameron Dicker|LAC|K|141.0
15       Curtis Samuel|Wsh|WR|137.0
Name: Team Stassen, dtype: object

In [71]:
# Projected points with roster breakdown
for col in final_teams.columns:
    print(col, final_teams[col].apply(lambda x: x.points).sum())
    print(final_teams[col].apply(lambda x: x.position).value_counts())

Team 1 2802.1000000000004
Team 1
WR    8
RB    3
QB    2
TE    1
D     1
K     1
Name: count, dtype: int64
Team 2 2975.7
Team 2
RB    6
WR    5
K     2
QB    1
TE    1
D     1
Name: count, dtype: int64
Team 3 2932.1
Team 3
WR    6
RB    3
TE    2
QB    2
D     2
K     1
Name: count, dtype: int64
Team 4 2853.6
Team 4
WR    8
RB    4
QB    1
TE    1
D     1
K     1
Name: count, dtype: int64
Team 5 2872.6000000000004
Team 5
WR    7
RB    5
QB    2
TE    1
D     1
Name: count, dtype: int64
Team 6 2829.6000000000004
Team 6
WR    7
RB    4
TE    2
QB    1
D     1
K     1
Name: count, dtype: int64
Team 7 2900.3
Team 7
WR    5
RB    5
QB    2
TE    2
D     1
K     1
Name: count, dtype: int64
Team 8 2706.2000000000003
Team 8
WR    6
RB    6
TE    1
QB    1
K     1
D     1
Name: count, dtype: int64
Team Stassen 3078.6000000000004
Team Stassen
RB    5
WR    5
TE    2
QB    2
D     1
K     1
Name: count, dtype: int64
Team 10 2758.5
Team 10
RB    6
WR    6
QB    1
TE    1
D     1
K     1
Name: coun